In [1]:
%run 'scripts/my_utils.py'
%run 'scripts/gen_data.py' --confirm 'no'
%run 'scripts/dataset.py'

In [2]:

import torch
import gc , random
import numpy as np
from scripts import environ
from scripts import my_utils as my_utils
from scripts.function import *
from scripts.environ import get_config
from scripts.dataset import ModelData
config = get_config()

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
storage_loader = my_utils.versatile_storage(config['STORAGE_TYPE'])
process_name = 'train'
num_states = 3

class container():
    def __init__(self) -> None:
        pass

data = ModelData('day' , config)
ShareNames = container()
ShareNames.model_date_list = data.model_date_list
ShareNames.test_full_dates = data.test_full_dates
data2 = data

In [3]:
import scripts.special.TRA as TRA
import torch
import torch.nn as nn

In [7]:
config['TRA_switch'] = True #affect 1.buffer includes hist_loss 2.seqlen includes labels of a big number
tra_num_states = 3 if config['TRA_switch'] else 1
data = data2
data2.buffer_functions(init = TRA.buffer_init(tra_num_states) , process = TRA.buffer_process(tra_num_states))
data2.create_dataloader('train' , 'train' , ShareNames.model_date_list[0] , config['MODEL_PARAM']['seqlens'][0])

net = TRA.TRA(nn.LSTM(6,16,1,batch_first=True),16,tra_num_states).cuda()

In [8]:


loss_train , loss_valid , ic_train , ic_valid = [] , [] , [] , []
iter_train , iter_valid = data2.dataloaders['train'] , data2.dataloaders['valid']

net.train()
optimizer = torch.optim.Adam(net.parameters())

modifier = {
    'inputs': lambda inputs,batch,data:inputs,
    'metric': lambda metric,batch,data:metric,
    'update': lambda batch,data:None
}
if config['TRA_switch']:
    modifier['inputs'] = lambda inputs,batch,data:net.modifier_inputs(inputs,batch,data)
    modifier['metric'] = lambda metric,batch,data:net.modifier_metric(metric,batch,data)
    modifier['update'] = lambda batch,data:net.modifier_update(batch,data)

k = 'hist_loss'
a_train , a_valid = [] , []

for ii in range(10):
    l_train , l_valid = [] , []
    net.train()   
    for _ , batch_data in enumerate(iter_train):
        x , y , w , i = tuple([batch_data[k] for k in ['x','y','w','i']])
        # d , rw = data2.buffer[k] , data2.seqs[k]
        inputs = modifier['inputs'](x , batch_data , data2)
        optimizer.zero_grad()
        
        pred , hidden = net(inputs)
        metric = {'loss':-pearson(pred , y) , 'score':pearson(pred , y).item()}
        metric = modifier['metric'](metric , batch_data , data2)
        
        metric['loss'].backward()
        optimizer.step()
        
        modifier['update'](batch_data , data2)

        l_train.append(metric['loss'].item())

    net.eval()     
    for i , batch_data in enumerate(iter_valid):
        x , y , w , i = tuple([batch_data[k] for k in ['x','y','w','i']])
        inputs = modifier['inputs'](x , batch_data , data2)
        pred , _ = net(inputs)
        metric = {'loss':-pearson(pred , y) , 'score':pearson(pred , y).item()}
        metric = modifier['metric'](metric , batch_data , data2)
        modifier['update'](batch_data , data2)
        l_valid.append(metric['score'])

    print(-np.mean(l_train) , np.mean(l_valid))



0.0018562273901291003 -5.481716361828148e-05
0.0116781241951095 0.02863265248015523
0.029439556947280024 0.03618217743933201
0.035790456572195165 0.04031315818428993
0.038990421732730304 0.04690311327576637
0.040948511220867165 0.040868749283254145
0.04401870260669573 0.049782258458435535
0.04882481971101181 0.05145879816263914
0.05129135420193543 0.05217417702078819
0.05393777334609547 0.05445307847112417
